In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit, OptimizeWarning
from sklearn.metrics import r2_score
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

In [2]:
fig_folder = 'E:\\projects_q_30_10_2024\\isttc\\results\\monkey\\fig_drafts\\'

### Get the data

In [3]:
results_folder = 'E:\\projects_q_30_10_2024\\isttc\\results\\monkey\\fixation_period_1000ms\\'

pfdl_acf_average_trial_pearsonr_df = pd.read_pickle(results_folder + 'binned\\pfdl\\acf\\' + 'acf_average_trial_pearsonr_constrained_input_50ms_20lags_df.pkl')
pfp_acf_average_trial_pearsonr_df = pd.read_pickle(results_folder + 'binned\\pfp\\acf\\' + 'acf_average_trial_pearsonr_constrained_input_50ms_20lags_df.pkl')

pfdl_acf_average_trial_sttc_df_ = pd.read_pickle(results_folder + 'non_binned\\pfdl\\acf\\' + 'acf_average_trial_sttc_50ms_20lags_df.pkl')
pfp_acf_average_trial_sttc_df_ = pd.read_pickle(results_folder + 'non_binned\\pfp\\acf\\' + 'acf_average_trial_sttc_50ms_20lags_df.pkl')

pfdl_constrained_data_acf_binned_1000_df = pd.read_pickle(results_folder + 'pfdl_constrained_data_acf_binned_1000_df.pkl')
pfp_constrained_data_acf_binned_1000_df = pd.read_pickle(results_folder + 'pfp_constrained_data_acf_binned_1000_df.pkl')

pfdl_included_units_l = pfdl_constrained_data_acf_binned_1000_df.query('n_trials_more_20 == True and no_zero_bins == 1 and one_hz_activity == 1')['unit_id'].values
pfdl_acf_average_trial_sttc_df = pfdl_acf_average_trial_sttc_df_[pfdl_acf_average_trial_sttc_df_['unit_id'].isin(pfdl_included_units_l)].copy()
pfdl_acf_average_trial_sttc_df.reset_index(inplace=True, drop=True)

pfp_included_units_l = pfp_constrained_data_acf_binned_1000_df.query('n_trials_more_20 == True and no_zero_bins == 1 and one_hz_activity == 1')['unit_id'].values
pfp_acf_average_trial_sttc_df = pfp_acf_average_trial_sttc_df_[pfp_acf_average_trial_sttc_df_['unit_id'].isin(pfp_included_units_l)].copy()
pfp_acf_average_trial_sttc_df.reset_index(inplace=True, drop=True)

print('N neurons in PFdl {} \nN neurons in PFp {}'.format(len(pfdl_acf_average_trial_pearsonr_df), len(pfp_acf_average_trial_pearsonr_df)))
print('N neurons in PFdl {} \nN neurons in PFp {}'.format(len(pfdl_acf_average_trial_sttc_df), len(pfp_acf_average_trial_sttc_df)))

print('NaNs in PFdl acf {}'.format(pfdl_acf_average_trial_pearsonr_df.isnull().any().any()))
print('NaNs in PFp acf {}'.format(pfp_acf_average_trial_pearsonr_df.isnull().any().any()))

print('NaNs in PFdl acf {}'.format(pfdl_acf_average_trial_sttc_df.isnull().any().any()))
print('NaNs in PFp acf {}'.format(pfp_acf_average_trial_sttc_df.isnull().any().any()))

N neurons in PFdl 367 
N neurons in PFp 318
N neurons in PFdl 367 
N neurons in PFp 318
NaNs in PFdl acf False
NaNs in PFp acf False
NaNs in PFdl acf False
NaNs in PFp acf False


In [4]:
pfdl_acf_average_trial_sttc_df.head(2)

,unit_id,acf_0,acf_1,acf_2,acf_3,acf_4,acf_5,acf_6,acf_7,acf_8,...,acf_10,acf_11,acf_12,acf_13,acf_14,acf_15,acf_16,acf_17,acf_18,acf_19
0,0,1.0,0.559754,0.515696,0.502268,0.482715,0.449641,0.427961,0.404055,0.427110,...,0.403228,0.371616,0.418338,0.383152,0.388130,0.381526,0.335855,0.340939,0.316042,0.353275
1,3,1.0,0.437931,0.435684,0.405248,0.370371,0.353249,0.377837,0.349305,0.373409,...,0.336337,0.316071,0.348698,0.319215,0.297623,0.314771,0.261334,0.313641,0.297711,0.332270


### Calc ACF and tau constraints

1. ACF decline within 150-250ms
2. tau > 0

In [9]:
def calculate_acf_decline_flag(acf_2d, start_idx=3, end_idx=5):
    decay_l = []
    for i in range(acf_2d.shape[0]):
        decay_l.append(np.all(np.diff(acf_2d[i, start_idx:end_idx]) <= 0))
    return decay_l

In [17]:
acf_cols = ['acf_' + str(i) for i in range(20)]
pfdl_sttc_2d = pfp_acf_average_trial_sttc_df[acf_cols].values
pfdl_sttc_2d_acf_decay = calculate_acf_decline_flag(pfdl_sttc_2d, start_idx=3, end_idx=5)

pfp_acf_average_trial_sttc_df.insert(1, 'decline_150_250', pfdl_sttc_2d_acf_decay)
pfp_acf_average_trial_sttc_df

,unit_id,decline_150_250,acf_0,acf_1,acf_2,acf_3,acf_4,acf_5,acf_6,acf_7,...,acf_10,acf_11,acf_12,acf_13,acf_14,acf_15,acf_16,acf_17,acf_18,acf_19
0,0,False,1.0,0.149503,0.090789,0.107328,0.142620,0.069971,0.040691,0.019864,...,0.037815,0.014939,0.067810,0.035148,0.071576,0.086282,0.092043,0.015594,0.023097,-0.027668
1,7,True,1.0,0.182435,0.124165,0.144258,0.092535,0.039624,0.032018,0.003664,...,0.013006,0.123663,0.049586,0.099034,0.117421,-0.030952,0.198415,-0.025379,-0.018034,-0.010345
2,8,True,1.0,0.023271,-0.001727,0.060637,0.047026,0.036058,0.047103,0.007462,...,0.044325,0.027195,-0.025914,0.085759,0.000219,-0.027425,-0.026469,0.069228,0.117760,-0.012500
3,13,True,1.0,0.094323,0.096056,0.060490,0.015418,0.058971,0.041777,0.041605,...,0.031403,0.032195,0.012216,0.033636,-0.008720,0.041285,-0.022976,0.013552,-0.036138,-0.036716
4,14,False,1.0,0.304562,0.282768,0.266142,0.277202,0.235367,0.239858,0.218427,...,0.226230,0.208726,0.274993,0.253289,0.198503,0.312302,0.245443,0.229478,0.296903,0.081534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,533,True,1.0,0.190317,0.170144,0.137822,0.049940,0.059929,0.076844,0.063933,...,0.036812,0.007788,0.032474,0.054677,0.078630,-0.005211,0.059345,-0.025377,-0.021156,-0.015678
314,535,False,1.0,0.169368,0.100475,0.080767,0.166638,0.083375,0.093231,0.057848,...,0.087249,0.077334,0.028804,0.009610,0.027736,0.005629,-0.000898,0.000650,0.040653,-0.036332
315,536,True,1.0,0.697167,0.584355,0.548639,0.519257,0.508686,0.509922,0.495104,...,0.479326,0.469536,0.475691,0.438489,0.476580,0.486160,0.417150,0.432794,0.460129,0.545915
316,537,True,1.0,0.353760,0.303956,0.212869,0.142538,0.133645,0.123950,0.133067,...,0.152806,0.190832,0.176680,0.178352,0.114828,0.137761,0.132963,0.097245,0.137487,0.075828


In [18]:
pfdl_pearsonr_2d = pfp_acf_average_trial_pearsonr_df[acf_cols].values
pfdl_pearsonr_2d_acf_decay = calculate_acf_decline_flag(pfdl_pearsonr_2d, start_idx=3, end_idx=5)

pfp_acf_average_trial_pearsonr_df.insert(1, 'decline_150_250', pfdl_pearsonr_2d_acf_decay)
pfp_acf_average_trial_pearsonr_df

,unit_id,decline_150_250,acf_0,acf_1,acf_2,acf_3,acf_4,acf_5,acf_6,acf_7,...,acf_10,acf_11,acf_12,acf_13,acf_14,acf_15,acf_16,acf_17,acf_18,acf_19
0,0,False,1.0,0.119814,0.063838,0.059544,0.111271,0.030454,-0.016651,0.004328,...,0.006648,-0.061097,0.015050,-0.035423,-0.007152,0.047979,0.046246,-0.044815,0.083069,-0.100631
1,7,True,1.0,0.165524,0.143937,0.136925,0.076128,0.047678,0.020910,-0.051451,...,-0.056199,0.049991,-0.019161,0.078554,0.090126,-0.085721,0.058843,-0.065524,-0.047686,-0.029775
2,8,True,1.0,0.016236,-0.052774,0.028938,0.024231,0.019720,-0.005943,0.004207,...,0.013200,-0.008908,-0.079066,0.024144,-0.018514,-0.054987,-0.072225,0.077053,0.162551,-0.038462
3,13,True,1.0,0.108287,0.080578,0.034018,-0.021056,0.022848,0.001407,-0.007136,...,-0.019022,-0.015395,-0.039104,-0.018281,-0.041328,0.001767,-0.075446,-0.054545,-0.053715,-0.085394
4,14,True,1.0,0.188664,0.188402,0.189508,0.182732,0.138364,0.098417,0.113226,...,0.083071,0.051279,0.086673,0.120306,0.061080,0.236563,0.133188,0.127749,0.145032,-0.121536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,533,True,1.0,0.227004,0.128809,0.089402,0.016421,0.010858,0.004295,0.030179,...,-0.024318,-0.020802,-0.029242,0.042348,0.070263,-0.038270,0.001607,-0.062297,-0.049624,-0.044080
314,535,False,1.0,0.202892,0.102934,0.049968,0.171397,0.084365,0.076613,0.049525,...,0.064029,0.059569,0.018589,-0.039327,-0.007137,-0.067098,-0.046897,-0.027033,-0.049110,-0.099247
315,536,True,1.0,0.609315,0.394260,0.282607,0.191286,0.138722,0.160405,0.159635,...,0.181895,0.133341,0.083622,0.070694,0.112729,0.176730,0.176457,0.151898,0.323771,0.162539
316,537,True,1.0,0.339044,0.212507,0.150041,0.078393,0.046208,0.064723,0.055259,...,0.071213,0.085362,0.067469,0.095290,0.025186,0.042913,0.046379,-0.005985,0.038395,-0.067896


In [19]:
np.count_nonzero(pfdl_pearsonr_2d_acf_decay)

228

In [20]:
np.count_nonzero(pfdl_sttc_2d_acf_decay)

217